In [20]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [23]:
dataset = datasets.MNIST(root='../data', train=True, download=True, transform=None)

In [6]:
len(mnist_train)

60000

In [19]:
def to_img(x):
    x = x.view(x.size(0), 1, 28, 28)
    return x

def plot_sample_img(img, name):
    img = img.view(1, 28, 28)
    save_image(img, './sample_{}.png'.format(name))


def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor

def tensor_round(tensor):
    return torch.round(tensor)

In [22]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda tensor:min_max_normalization(tensor, 0, 1)),
    transforms.Lambda(lambda tensor:tensor_round(tensor))
])

In [33]:
batch_size = 128
learning_rate = 1e-3
num_epochs = 200

In [40]:
dataset = MNIST('../data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [30]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        
        self.Encoder = nn.Sequential(
            nn.Linear(28 * 28, 256),
            nn.ReLU(True),
            nn.Linear(256, 64),
            nn.ReLU(True))
        
        self.Decoder = nn.Sequential(
            nn.Linear(64, 256),
            nn.ReLU(True),
            nn.Linear(256, 28*28),
            nn.Sigmoid())
        
        def forward(self, x):
            x = self.Encoder(x)
            x = self.Decoder(x)
            return x

In [34]:
model = AutoEncoder()

In [35]:
criterion = nn.BCELoss()

In [36]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [43]:
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        
        # forward
        output = model(img)        
        loss = criterion(output, img)
        MSE_loss = nn.MSELoss()(output, img)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # log
        print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
            .format(epoch + 1, num_epochs, loss.data[0], MSE_loss.data[0]))
        if epoch % 10 == 0:
            x = to_img(img.cpu().data)
            x_hat = to_img(output.cpu().data)
            save_image(x, './mlp_img/x_{}.png'.format(epoch))
            save_image(x_hat, './mlp_img/x_hat_{}.png'.format(epoch))


NotImplementedError: 

In [51]:
import os

import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')


def to_img(x):
    x = x.view(x.size(0), 1, 28, 28)
    return x

num_epochs = 200
batch_size = 128
learning_rate = 1e-3


def plot_sample_img(img, name):
    img = img.view(1, 28, 28)
    save_image(img, './sample_{}.png'.format(name))


def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor


def tensor_round(tensor):
    return torch.round(tensor)

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda tensor:min_max_normalization(tensor, 0, 1)),
    transforms.Lambda(lambda tensor:tensor_round(tensor))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 256),
            nn.ReLU(True),
            nn.Linear(256, 64),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(64, 256),
            nn.ReLU(True),
            nn.Linear(256, 28 * 28),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder() # .cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img) #.cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        MSE_loss = nn.MSELoss()(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print(f'epoch [{epoch+1}/{num_epochs}], loss: {loss}')
    #print(f'epoch [{epoch+1}/{num_epochs}], loss:{loss.data[0]}, MSE_loss:{MSE_loss.data[0]}')
    if epoch % 10 == 0:
        x = to_img(img.cpu().data)
        x_hat = to_img(output.cpu().data)
        save_image(x, './mlp_img/x_{}.png'.format(epoch))
        save_image(x_hat, './mlp_img/x_hat_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')

epoch [1/200], loss: 0.10835842043161392
epoch [2/200], loss: 0.07667386531829834
epoch [3/200], loss: 0.06962911039590836
epoch [4/200], loss: 0.06037367507815361
epoch [5/200], loss: 0.05657651647925377
epoch [6/200], loss: 0.051107924431562424
epoch [7/200], loss: 0.05318485200405121
epoch [8/200], loss: 0.05057459697127342
epoch [9/200], loss: 0.0432075634598732
epoch [10/200], loss: 0.04587759077548981
epoch [11/200], loss: 0.043028824031353
epoch [12/200], loss: 0.03823643922805786
epoch [13/200], loss: 0.0371742881834507
epoch [14/200], loss: 0.039610810577869415
epoch [15/200], loss: 0.037864647805690765
epoch [16/200], loss: 0.03420212119817734
epoch [17/200], loss: 0.033305563032627106
epoch [18/200], loss: 0.03279121592640877
epoch [19/200], loss: 0.033255722373723984
epoch [20/200], loss: 0.03365696594119072
epoch [21/200], loss: 0.03221025690436363
epoch [22/200], loss: 0.030603129416704178
epoch [23/200], loss: 0.035512421280145645
epoch [24/200], loss: 0.0342829413712024

epoch [192/200], loss: 0.023518387228250504
epoch [193/200], loss: 0.02358437143266201
epoch [194/200], loss: 0.022740941494703293
epoch [195/200], loss: 0.02223658189177513
epoch [196/200], loss: 0.024499384686350822
epoch [197/200], loss: 0.0260175671428442
epoch [198/200], loss: 0.022897666320204735
epoch [199/200], loss: 0.022216806188225746
epoch [200/200], loss: 0.023668775334954262


In [ ]:
1+1